# Medical Image Classification
## Solution 1: Building a Convnet from Scratch

In this exercise, we will build a classifier model from scratch that is able to distinguish medical images. We will follow these steps:

0. Prepare the data & model hyperparameters
1. Explore the example data
2. Build a small convnet from scratch to solve our classification problem
3. Evaluate training and validation accuracy

Let's go!

##Prepare data to classicfication classes

In [5]:
import os
import shutil
from glob import glob
import random


def shuffle2(data):
    """shuffle elements in data, but not in place"""
    data2 = data[:]
    random.shuffle(data2)
    return data2


def get_subdirs(base_dir):
    """get subdirectories"""
    return [i for i in glob(base_dir + "/*/") if os.path.isdir(i)]


def get_orig_struct(base_dir):
    """
    Represent original file structure in a dictionary
    """
    assert os.path.isdir(base_dir)
    sub_dirs = get_subdirs(base_dir)
    original_structure = {subd: os.listdir(subd) for subd in sub_dirs}
    return original_structure


def make_train_test_dict(orig_struct_dict, test_prop):
    """
    Create dictionary representing train/test file structure
    """
    assert isinstance(orig_struct_dict, dict)
    new_struct = {"training": {}, "validation": {}}
    for key, value in orig_struct_dict.items():
        train_dat, test_dat = train_test_split(data=value, test_prop=test_prop)
        new_struct["training"].update({key: train_dat})
        new_struct["validation"].update({key: test_dat})
    return new_struct


def train_test_split(data, test_prop):
    """docstring"""
    n_test = round(len(data) * test_prop)
    n_train = len(data) - n_test
    data = shuffle2(data)
    train = data[:n_train]
    test = data[-n_test:]
    # check nothing's been lost
    assert len(train) + len(test) == len(data)
    return train, test


def get_label(filename):
    """
    get label from base_dir filename as seen in the dictionaries
    e.g if the key is "example/data/apples/"
        this will return "apples"
    """
    return os.path.basename(os.path.normpath(filename))


def check_dir(filename):
    """
    Given a path, checks the sub-directories exists,
    if not then tries to create them
    """
    subdir_path = os.path.dirname(filename)
    try:
        os.makedirs(subdir_path)
    except OSError:
        if os.path.isdir(subdir_path):
            pass
        else:
            err_msg = "failed to create directory {}".format(subdir_path)
            raise RuntimeError(err_msg)


def create_path_lists(source, destination, test_prop):
    """
    Return two lists, the original filepaths, and the destination filepaths
    Idea is to use shutil.copy2(source, destination) on each element of the 
    paired lists.
    """
    orig_struct = get_orig_struct(base_dir=source)
    train_test_dict = make_train_test_dict(orig_struct, test_prop)
    original_path = []
    new_path = []
    for tt, subdict in train_test_dict.items():
        for class_labels, files in subdict.items():
            orig = [os.path.join(class_labels, i) for i in files]
            dest = [os.path.join(destination, tt, get_label(class_labels), i) for i in files]
            original_path.extend(orig)
            new_path.extend(dest)
    return original_path, new_path


def train_test_split_dir(source, destination, test_prop=0.3):
    original_paths, new_paths = create_path_lists(
        source=source, destination=destination, test_prop=test_prop)
    for i, j in zip(original_paths, new_paths):
        check_dir(j) # check if destination directory exists, create if needed
shutil.copy2(i, j)
#train_test_split_dir(source=base_dir, destination=target_dir)
#train_test_split_dir(source=base_dir, destination=target_dir, test_prop=0.2)
#base_dir = target_dir

NameError: name 'i' is not defined

In [2]:
#output dimension = 15
class_names = ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Edema', 'Effusion', 'Emphysema', 'Fibrosis', 'Hernia', 'Infiltration', 'Mass', 'No Finding', 'Nodule', 'Pleural_Thickening', 'Pneumonia', 'Pneumothorax' ]
#print 'You have %d image categories.' % len(class_names)
img_width = 1024.
img_height = 1024
color_channels = 3
aspect_ratio = img_width/img_height  #4:3
#print 'The image aspect ratio (width divided by height) is %f.' %aspect_ratio

##Set Model Hyperparameters

*   Maximale Bildgröße
*   Training Parameter



In [6]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
base_dir = '/home/ente/Schreibtisch/2018 - 11 - sorted data'
target_dir  = '/home/ente/Schreibtisch/data split'
max_pixels = 299
target_width = max_pixels
target_height = max_pixels # 768*1.33 = 224 too keep aspect_ratio
pooling_window = 2
conv_window = 3
kernel_size = 16
activation = "relu"
optimizer = 'Adam'
loss='categorical_crossentropy'
#metrics= ['acc']
metrics=['categorical_accuracy']
class_mode = 'categorical'
batch_size = 20
epochs = 15
verbose = 2
lr = 0.001

## Explore the Example Data

Let's start by exploring our example data, they are locally stored in `/home/ente/Schreibtisch/`.

The base directory `/Users/mkunzmann/Desktop/sorted data/`, which contains `train` and `validation` subdirectories for the training and validation datasets (see the [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/validation/check-your-intuition) for a refresher on training, validation, and test sets), which in turn each contain `subclass` subdirectories. Let's define each of these directories:

In [7]:
train_test_split_dir(source=base_dir, destination=target_dir)
base_dir = target_dir

import os
train_dir = os.path.join(base_dir, 'training')
validation_dir = os.path.join(base_dir, 'validation')
# Directories with our training pictures
train_ate_dir = os.path.join(train_dir, class_names[0])
train_car_dir = os.path.join(train_dir, class_names[1])
train_con_dir = os.path.join(train_dir, class_names[2])
train_ede_dir = os.path.join(train_dir, class_names[3])
train_eff_dir = os.path.join(train_dir, class_names[4])

train_emp_dir = os.path.join(train_dir, class_names[5])
train_fib_dir = os.path.join(train_dir, class_names[6])
train_her_dir = os.path.join(train_dir, class_names[7])
train_inf_dir = os.path.join(train_dir, class_names[8])
train_mas_dir = os.path.join(train_dir, class_names[9])

train_nof_dir = os.path.join(train_dir, class_names[10])
train_nod_dir = os.path.join(train_dir, class_names[11])
train_ple_dir = os.path.join(train_dir, class_names[12])
train_pne_dir = os.path.join(train_dir, class_names[13])
train_pn2_dir = os.path.join(train_dir, class_names[14])

# Directories with our validation pictures
validation_ate_dir = os.path.join(validation_dir, class_names[0])
validation_car_dir = os.path.join(validation_dir, class_names[1])
validation_con_dir = os.path.join(validation_dir, class_names[2])
validation_ede_dir = os.path.join(validation_dir, class_names[3])
validation_eff_dir = os.path.join(validation_dir, class_names[4])

validation_emp_dir = os.path.join(validation_dir, class_names[5])
validation_fib_dir = os.path.join(validation_dir, class_names[6])
validation_her_dir = os.path.join(validation_dir, class_names[7])
validation_inf_dir = os.path.join(validation_dir, class_names[8])
validation_mas_dir = os.path.join(validation_dir, class_names[9])

validation_nof_dir = os.path.join(validation_dir, class_names[10])
validation_nod_dir = os.path.join(validation_dir, class_names[11])
validation_ple_dir = os.path.join(validation_dir, class_names[12])
validation_pne_dir = os.path.join(validation_dir, class_names[13])
validation_pn2_dir = os.path.join(validation_dir, class_names[14])


Now, let's see what the filenames look like in the `cats` and `dogs` `train` directories (file naming conventions are the same in the `validation` directory):

In [8]:
train_ate_fnames = os.listdir(train_ate_dir)
train_car_fnames = os.listdir(train_car_dir)
train_con_fnames = os.listdir(train_con_dir)
train_ede_fnames = os.listdir(train_ede_dir)
train_eff_fnames = os.listdir(train_eff_dir)

train_emp_fnames = os.listdir(train_emp_dir)
train_fib_fnames = os.listdir(train_fib_dir)
train_her_fnames = os.listdir(train_her_dir)
train_inf_fnames = os.listdir(train_inf_dir)
train_mas_fnames = os.listdir(train_mas_dir)

train_nof_fnames = os.listdir(train_nof_dir)
train_nod_fnames = os.listdir(train_nod_dir)
train_ple_fnames = os.listdir(train_ple_dir)
train_pne_fnames = os.listdir(train_pne_dir)
train_pn2_fnames = os.listdir(train_pn2_dir)

print (train_ate_fnames[:10])
train_car_fnames.sort()
print (train_car_fnames[:10])


[]
[]


Let's find out the total number of all class-images in the `train` and `validation` directories:

In [9]:
print ('total train/validation Atelectasis images:       ', len(os.listdir(train_ate_dir)), '/', len(os.listdir(validation_ate_dir))) 
print ('total train/validation Cardiomegaly images:      ', len(os.listdir(train_car_dir)), '/', len(os.listdir(validation_car_dir))) 
print ('total train/validation Consolidation images:     ', len(os.listdir(train_con_dir)), '/', len(os.listdir(validation_con_dir))) 
print ('total train/validation Edema images:             ', len(os.listdir(train_ede_dir)), '/', len(os.listdir(validation_ede_dir))) 
print ('total train/validation Effusion images:          ', len(os.listdir(train_eff_dir)), '/', len(os.listdir(validation_eff_dir))) 

print ('total train/validation Emphysema images:         ', len(os.listdir(train_emp_dir)), '/', len(os.listdir(validation_emp_dir))) 
print ('total train/validation Fibrosis images:          ', len(os.listdir(train_fib_dir)), '/', len(os.listdir(validation_fib_dir))) 
print ('total train/validation Hernia images:            ', len(os.listdir(train_her_dir)), '/', len(os.listdir(validation_her_dir))) 
print ('total train/validation Infiltration images:      ', len(os.listdir(train_inf_dir)), '/', len(os.listdir(validation_inf_dir))) 
print ('total train/validation Mass images:              ', len(os.listdir(train_mas_dir)), '/', len(os.listdir(validation_mas_dir))) 

print ('total train/validation No_Finding images:        ', len(os.listdir(train_nof_dir)), '/', len(os.listdir(validation_nof_dir))) 
print ('total train/validation Nodule images:            ', len(os.listdir(train_nod_dir)), '/', len(os.listdir(validation_nod_dir))) 
print ('total train/validation Pleural_Thickening images:', len(os.listdir(train_ple_dir)), '/', len(os.listdir(validation_ple_dir))) 
print ('total train/validation Pneumonia images:         ', len(os.listdir(train_pne_dir)), '/', len(os.listdir(validation_pne_dir))) 
print ('total train/validation Pneumothorax images:      ', len(os.listdir(train_pn2_dir)), '/', len(os.listdir(validation_pn2_dir))) 

total train/validation Atelectasis images:        0 / 0
total train/validation Cardiomegaly images:       0 / 0
total train/validation Consolidation images:      0 / 0
total train/validation Edema images:              0 / 0
total train/validation Effusion images:           0 / 0
total train/validation Emphysema images:          0 / 0
total train/validation Fibrosis images:           0 / 0
total train/validation Hernia images:             0 / 0
total train/validation Infiltration images:       0 / 0
total train/validation Mass images:               0 / 0
total train/validation No_Finding images:         0 / 0
total train/validation Nodule images:             0 / 0
total train/validation Pleural_Thickening images: 0 / 0
total train/validation Pneumonia images:          0 / 0
total train/validation Pneumothorax images:       0 / 0


For both cats and dogs, we have 1,000 training images and 500 test images.

Now let's take a look at a few pictures to get a better sense of what the cat and dog datasets look like. First, configure the matplot parameters:

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Parameters for our graph; we'll output images in a 15x4 configuration
nrows = 15
#nrows = 2
ncols = 4

# Index for iterating over images
pic_index = 0


Now, display a batch of 8 cat and 8 dog pictures. You can rerun the cell to see a fresh batch each time:

In [ ]:
# Set up matplotlib fig, and size it to fit 4x4 pics
fig = plt.gcf()
fig.set_size_inches(ncols * 4, nrows * 4)

pic_index += 8
next_ate_pix = [os.path.join(train_ate_dir, fname) 
                for fname in train_ate_fnames[pic_index-8:pic_index]]
next_car_pix = [os.path.join(train_car_dir, fname) 
                for fname in train_car_fnames[pic_index-8:pic_index]]

for i, img_path in enumerate(next_ate_pix+next_car_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()


## Building a Small Convnet from Scratch to Get to 72% Accuracy

The images that will go into our convnet are 1024x1024 color images (in the next section on Data Preprocessing, we'll add handling to resize all the images to 299x299 before feeding them into the neural network).

Let's code up the architecture. We will stack 3 {convolution + relu + maxpooling} modules. Our convolutions operate on 3x3 windows and our maxpooling layers operate on 2x2 windows. Our first convolution extracts 16 filters, the following one extracts 32 filters, and the last one extracts 64 filters.

**NOTE**: This is a configuration that is widely used and known to work well for image classification. Also, since we have relatively few training examples (1,000), using just three convolutional modules keeps the model small, which lowers the risk of overfitting (which we'll explore in more depth in Exercise 2.)

In [10]:
from tensorflow.keras import layers
from tensorflow.keras import Model

In [11]:
# Our input feature map is 150x150x3: 150x150 for the image pixels, and 3 for
# the three color channels: R, G, and B
#img_input = layers.Input(shape=(150, 150, 3))
img_input = layers.Input(shape=(img_width, img_height, color_channels))

# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(kernel_size, conv_window, activation=activation)(img_input)
x = layers.MaxPooling2D(pooling_window)(x)

# Second convolution extracts 32 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(2*kernel_size, conv_window, activation=activation)(x)
x = layers.MaxPooling2D(pooling_window)(x)

# Third convolution extracts 64 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
x = layers.Conv2D(4*kernel_size, conv_window, activation=activation)(x)
x = layers.MaxPooling2D(pooling_window)(x)

On top of it we stick two fully-connected layers. Because we are facing a two-class classification problem, i.e. a *binary classification problem*, we will end our network with a [*sigmoid* activation](https://wikipedia.org/wiki/Sigmoid_function), so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).

In [12]:
# Flatten feature map to a 1-dim tensor so we can add fully connected layers
x = layers.Flatten()(x)

# Create a fully connected layer with ReLU activation and 512 hidden units
x = layers.Dense(512, activation=activation)(x)

# Create output layer with a single node and sigmoid activation
#output = layers.Dense(1, activation='sigmoid')(x)
output = layers.Dense(len(class_names), activation = 'softmax') (x)


# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)

Let's summarize the model architecture:

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1024, 1024, 3)     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 1022, 1022, 16)    448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 511, 511, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 509, 509, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 126, 126, 64)      0         
__________

The "output shape" column shows how the size of your feature map evolves in each successive layer. The convolution layers reduce the size of the feature maps by a bit due to padding, and each pooling layer halves the feature map.

Next, we'll configure the specifications for model training. We will train our model with the `binary_crossentropy` loss, because it's a binary classification problem and our final activation is a sigmoid. (For a refresher on loss metrics, see the [Machine Learning Crash Course](https://developers.google.com/machine-learning/crash-course/descending-into-ml/video-lecture).) We will use the `rmsprop` optimizer with a learning rate of `0.001`. During training, we will want to monitor classification accuracy.

**NOTE**: In this case, using the [RMSprop optimization algorithm](https://wikipedia.org/wiki/Stochastic_gradient_descent#RMSProp) is preferable to [stochastic gradient descent](https://developers.google.com/machine-learning/glossary/#SGD) (SGD), because RMSprop automates learning-rate tuning for us. (Other optimizers, such as [Adam](https://wikipedia.org/wiki/Stochastic_gradient_descent#Adam) and [Adagrad](https://developers.google.com/machine-learning/glossary/#AdaGrad), also automatically adapt the learning rate during training, and would work equally well here.)

In [14]:
#from tensorflow.keras.optimizers import RMSprop

#model.compile(loss='binary_crossentropy',
#              optimizer=RMSprop(lr=0.001),
#              metrics=['acc'])

model.compile(loss=loss,
              optimizer=optimizer,
              metrics=metrics)

### Data Preprocessing

Let's set up data generators that will read pictures in our source folders, convert them to `float32` tensors, and feed them (with their labels) to our network. We'll have one generator for the training images and one for the validation images. Our generators will yield batches of 20 images of size 150x150 and their labels (binary).

As you may already know, data that goes into neural networks should usually be normalized in some way to make it more amenable to processing by the network. (It is uncommon to feed raw pixels into a convnet.) In our case, we will preprocess our images by normalizing the pixel values to be in the `[0, 1]` range (originally all values are in the `[0, 255]` range).

In Keras this can be done via the `keras.preprocessing.image.ImageDataGenerator` class using the `rescale` parameter. This `ImageDataGenerator` class allows you to instantiate generators of augmented image batches (and their labels) via `.flow(data, labels)` or `.flow_from_directory(directory)`. These generators can then be used with the Keras model methods that accept data generators as inputs: `fit_generator`, `evaluate_generator`, and `predict_generator`.

In [15]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
# Flow training images in batches of 20 using train_datagen generator
train_datagen = ImageDataGenerator(rescale=1./255,
    #shear_range=0.2,
    #zoom_range=0.2,
    #horizontal_flip=True,
    validation_split=0.2) # set validation split

#test data will not be augmented
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(target_height, target_width),
    batch_size=batch_size,
    class_mode=class_mode,
    subset='training') # set as training data

validation_generator = test_datagen.flow_from_directory(
    #train_dir, # same directory as training data
    validation_dir,
    target_size=(target_height, target_width),
    batch_size=batch_size,
    class_mode=class_mode
    subset='validation') # set as validation data


SyntaxError: invalid syntax (<ipython-input-15-4851ae9106c6>, line 27)

### Training
Let's train on all 2,000 images available, for 15 epochs, and validate on all 1,000 test images. (This may take a few minutes to run.)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs = epochs)

### Visualizing Intermediate Representations

To get a feel for what kind of features our convnet has learned, one fun thing to do is to visualize how an input gets transformed as it goes through the convnet.

Let's pick a random cat or dog image from the training set, and then generate a figure where each row is the output of a layer, and each image in the row is a specific filter in that output feature map. Rerun this cell to generate intermediate representations for a variety of training images.

In [ ]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's define a new Model that will take an image as input, and will output
# intermediate representations for all layers in the previous model after
# the first.
successive_outputs = [layer.output for layer in model.layers[1:]]
visualization_model = Model(img_input, successive_outputs)

# Let's prepare a random input image of a cat or dog from the training set.
cat_img_files = [os.path.join(train_cats_dir, f) for f in train_cat_fnames]
dog_img_files = [os.path.join(train_dogs_dir, f) for f in train_dog_fnames]
img_path = random.choice(cat_img_files + dog_img_files)

img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)

# Rescale by 1/255
x /= 255

# Let's run our image through our network, thus obtaining all
# intermediate representations for this image.
successive_feature_maps = visualization_model.predict(x)

# These are the names of the layers, so can have them as part of our plot
layer_names = [layer.name for layer in model.layers]

# Now let's display our representations
for layer_name, feature_map in zip(layer_names, successive_feature_maps):
  if len(feature_map.shape) == 4:
    # Just do this for the conv / maxpool layers, not the fully-connected layers
    n_features = feature_map.shape[-1]  # number of features in feature map
    # The feature map has shape (1, size, size, n_features)
    size = feature_map.shape[1]
    # We will tile our images in this matrix
    display_grid = np.zeros((size, size * n_features))
    for i in range(n_features):
      # Postprocess the feature to make it visually palatable
      x = feature_map[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    # Display the grid
    scale = 20. / n_features
    plt.figure(figsize=(scale * n_features, scale))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

As you can see we go from the raw pixels of the images to increasingly abstract and compact representations. The representations downstream start highlighting what the network pays attention to, and they show fewer and fewer features being "activated"; most are set to zero. This is called "sparsity." Representation sparsity is a key feature of deep learning.


These representations carry increasingly less information about the original pixels of the image, but increasingly refined information about the class of the image. You can think of a convnet (or a deep network in general) as an information distillation pipeline.

### Evaluating Accuracy and Loss for the Model

Let's plot the training/validation accuracy and loss as collected during training:

In [ ]:
# Retrieve a list of accuracy results on training and test data
# sets for each training epoch
acc = history.history['acc']
val_acc = history.history['val_acc']

# Retrieve a list of list results on training and test data
# sets for each training epoch
loss = history.history['loss']
val_loss = history.history['val_loss']

# Get number of epochs
epochs = range(len(acc))

# Plot training and validation accuracy per epoch
plt.plot(epochs, acc)
plt.plot(epochs, val_acc)
plt.title('Training and validation accuracy')

plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

As you can see, we are **overfitting** like it's getting out of fashion. Our training accuracy (in blue) gets close to 100% (!) while our validation accuracy (in green) stalls as 70%. Our validation loss reaches its minimum after only five epochs.

Since we have a relatively small number of training examples (2000), overfitting should be our number one concern. Overfitting happens when a model exposed to too few examples learns patterns that do not generalize to new data, i.e. when the model starts using irrelevant features for making predictions. For instance, if you, as a human, only see three images of people who are lumberjacks, and three images of people who are sailors, and among them the only person wearing a cap is a lumberjack, you might start thinking that wearing a cap is a sign of being a lumberjack as opposed to a sailor. You would then make a pretty lousy lumberjack/sailor classifier.

Overfitting is the central problem in machine learning: given that we are fitting the parameters of our model to a given dataset, how can we make sure that the representations learned by the model will be applicable to data never seen before? How do we avoid learning things that are specific to the training data?

In the next exercise, we'll look at ways to prevent overfitting in the cat vs. dog classification model.

## Clean Up

Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [ ]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)